In [1]:
import numpy as np
import tensorflow as tf
import datetime
sess = tf.Session()

x = [[1,0,0,0,0],# 4*4  #h
    [0,1,0,0,0], #e
    [0,0,1,0,0], #l
    [0,0,1,0,0], #l
    [0,0,0,1,0]] #o

y = [[0,1,0,0,0], # 4*4 #e
    [0,0,1,0,0], #l
    [0,0,1,0,0], #l
    [0,0,0,1,0], #o
    [0,0,0,0,1]]

c_ = tf.zeros([5,5])
h_ = tf.zeros([5,5])


X = tf.placeholder(dtype=tf.float32, shape=[None, 5])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 5])
W = tf.Variable(tf.random_normal([5, 1]))*0.5
b = tf.Variable(tf.random_normal([1]))


#he
#el
#ll
#lo

seq_len = len(x) #len(x)cell 갯수, 인풋이 몇 덩어리인지
num_units = len(sess.run(c_))  #len(sess.run((c_)))# hiddenlayer


class lstm:
    def build(c, h):
        args = tf.concat((X,h), axis=1)
#        print(args)

        out_size = 4 * num_units
        proj_size = args.shape[-1]
#        print(out_size)
#        print(proj_size)

        weights = tf.ones([proj_size, out_size]) * 0.5
#        print(weights)


        out = tf.matmul(args, weights)
#        print(out)

        bias = tf.ones([out_size]) * 0.5
#        print(bias)

        concat = out + bias
#        print(concat)

        i, j, f, o = tf.split(concat, 4, 1)
#        print(i)
#        print(j)
#        print(f)
#        print(o)

        g = tf.tanh(j)
#        print(g)

        def sigmoid_array(x):
            return 1 / (1 + tf.exp(-x))

        forget_bias = 1.0

        sigmoid_f = sigmoid_array(f + forget_bias)
#        print(sigmoid_f)

        sigmoid_array(i) * g

        new_c = c * sigmoid_f + sigmoid_array(i) * g
#        print(new_c)

        new_h = tf.tanh(new_c) * sigmoid_array(o)
#        print(new_h)

#        print('\n new_h:',new_h)
#        print('\n new_c',new_c)

#        print(res[1].h)
#        print(res[1].c)

        return new_c, new_h

In [2]:
ta_c = tf.TensorArray(size=seq_len, dtype=tf.float32)
ta_h = tf.TensorArray(size=seq_len, dtype=tf.float32)

In [3]:
def body(last_state, last_output, step, ta_c, ta_h):
    
    output = lstm.build(last_state, last_output)[0]
    state = lstm.build(last_state, last_output)[1]
    ta_c = ta_c.write(step, state)
    ta_h = ta_h.write(step, output)
    return state, output, tf.add(step, 1), ta_c, ta_h
    

timesteps = seq_len

steps = lambda a, b, step, c, d: tf.less(step, timesteps)

lstm_output, lstm_state, step, ta_c, ta_h = tf.while_loop(steps, body, (c_, h_, 0, ta_c, ta_h), parallel_iterations=20)

In [11]:
output = lstm_output
logits = tf.matmul(output, W) + b

In [12]:
with tf.name_scope('mean_square_error'):
    mean_square_error = tf.reduce_sum(tf.square(tf.subtract(Y, tf.unstack(logits, axis = 1))))
tf.summary.scalar('mean_square_error', mean_square_error)

<tf.Tensor 'mean_square_error_3:0' shape=() dtype=string>

In [13]:
optimizer = tf.train.AdamOptimizer(0.00003)
minimize = optimizer.minimize(mean_square_error)

In [17]:
with tf.name_scope('error'):
    with tf.name_scope('mistakes'):
        mistakes = tf.not_equal(Y, tf.round(tf.unstack(logits, axis = 1)))
    with tf.name_scope('error'):
        error = tf.reduce_mean(tf.cast(mistakes, tf.float32))
tf.summary.scalar('error', error)

<tf.Tensor 'error_5:0' shape=() dtype=string>

In [24]:
sess = tf.Session()
merged = tf.summary.merge_all()

date = str(datetime.datetime.now())

init_op = tf.global_variables_initializer()
sess.run(init_op)

In [25]:
epoch = 50000

for i in range(epoch):
    if (i + 1) % 100 == 0:
        summary, incorrect, mean_squ_err = sess.run([merged, error, mean_square_error], {X:x, Y:y})
        
        print('Epoch {:4d} | incorrect {: 3.1f}% | mean squ error {: 3.1f}'.format(i + 1, incorrect * 100, mean_squ_err))
    else:
        summary, acc = sess.run([merged, error], {X:x, Y:y})


    sess.run(minimize,{X:x, Y:y})

Epoch  100 | incorrect  100.0% | mean squ error  96.2
Epoch  200 | incorrect  100.0% | mean squ error  95.2
Epoch  300 | incorrect  100.0% | mean squ error  94.2
Epoch  400 | incorrect  100.0% | mean squ error  93.3
Epoch  500 | incorrect  100.0% | mean squ error  92.3
Epoch  600 | incorrect  100.0% | mean squ error  91.4
Epoch  700 | incorrect  100.0% | mean squ error  90.4
Epoch  800 | incorrect  100.0% | mean squ error  89.5
Epoch  900 | incorrect  100.0% | mean squ error  88.6
Epoch 1000 | incorrect  100.0% | mean squ error  87.7
Epoch 1100 | incorrect  100.0% | mean squ error  86.8
Epoch 1200 | incorrect  100.0% | mean squ error  85.9
Epoch 1300 | incorrect  100.0% | mean squ error  85.0
Epoch 1400 | incorrect  100.0% | mean squ error  84.1
Epoch 1500 | incorrect  100.0% | mean squ error  83.2
Epoch 1600 | incorrect  100.0% | mean squ error  82.3
Epoch 1700 | incorrect  100.0% | mean squ error  81.5
Epoch 1800 | incorrect  100.0% | mean squ error  80.6
Epoch 1900 | incorrect  100.

Epoch 15400 | incorrect  80.0% | mean squ error  9.9
Epoch 15500 | incorrect  80.0% | mean squ error  9.7
Epoch 15600 | incorrect  80.0% | mean squ error  9.5
Epoch 15700 | incorrect  80.0% | mean squ error  9.3
Epoch 15800 | incorrect  80.0% | mean squ error  9.1
Epoch 15900 | incorrect  80.0% | mean squ error  8.9
Epoch 16000 | incorrect  80.0% | mean squ error  8.7
Epoch 16100 | incorrect  80.0% | mean squ error  8.6
Epoch 16200 | incorrect  80.0% | mean squ error  8.4
Epoch 16300 | incorrect  80.0% | mean squ error  8.2
Epoch 16400 | incorrect  80.0% | mean squ error  8.0
Epoch 16500 | incorrect  80.0% | mean squ error  7.9
Epoch 16600 | incorrect  80.0% | mean squ error  7.7
Epoch 16700 | incorrect  80.0% | mean squ error  7.6
Epoch 16800 | incorrect  80.0% | mean squ error  7.4
Epoch 16900 | incorrect  80.0% | mean squ error  7.3
Epoch 17000 | incorrect  80.0% | mean squ error  7.1
Epoch 17100 | incorrect  80.0% | mean squ error  7.0
Epoch 17200 | incorrect  80.0% | mean squ erro

KeyboardInterrupt: 